# Assignment #1 
# Part 2: LXD Automation ( 50 points )


## 1. Importing Images ( 10 points )
Import two container images using the `lxc image copy` command.
- Ubuntu 22.04 - jammy
- Debian 12 - bookworm

Hint: Use the bash command `lxc image copy images:$DISTRO/$VERSION local: --copy-aliases --auto-update`

### Testing

In [2]:
import pprint
from pylxd import Client

client = Client()

In [3]:
images = client.images.all()

In [15]:
for i in images:
    pprint.pprint(i.aliases)

[{'description': '', 'name': '22.04'},
 {'description': '', 'name': '22.04/amd64'},
 {'description': '', 'name': 'j'},
 {'description': '', 'name': 'j/amd64'},
 {'description': '', 'name': 'jammy'},
 {'description': '', 'name': 'jammy/amd64'}]
[{'description': '', 'name': 'debian/bookworm/default'},
 {'description': '', 'name': 'debian/bookworm/default/amd64'},
 {'description': '', 'name': 'debian/bookworm'},
 {'description': '', 'name': 'debian/bookworm/amd64'},
 {'description': '', 'name': 'debian/12/default'},
 {'description': '', 'name': 'debian/12/default/amd64'},
 {'description': '', 'name': 'debian/12'},
 {'description': '', 'name': 'debian/12/amd64'}]
[]


## 2. Creating Containers ( 10 points )
Create three containers named `www0`, `www1`, and `www2`.

Hint: https://pylxd.readthedocs.io/en/latest/instances.html

In [ ]:
# Write your code here
#created containers in terminal using lxc launch 

Display containers:

In [16]:
for i in client.containers.all():
    i.start()    
    print(f'{i.name}: {i.status}')

test0: Stopped
www0: Running
www1: Running
www2: Running


- We need to set the MTU of all containers to 1400 due to the VXLAN that your virtual server is on.
- I've written the code for this below, so all you have to do is run it.

In [4]:
!/snap/bin/lxc exec www0 -- ip link set eth0 mtu 1400
!/snap/bin/lxc exec www1 -- ip link set eth0 mtu 1400
!/snap/bin/lxc exec www2 -- ip link set eth0 mtu 1400

## 3. Installing a webserver ( 10 points )
For this section your will use the `pylxd` client update www[0:2] and install `apache2` with `apt`.

Be patient - updating all 3 containers can take a while.

In [5]:
# Write your code here
containers = ["www0", "www1", "www2"]
for container_name in containers:
    container = client.containers.get(container_name)
    print(f"updating {container_name}...")
    container.execute(["apt","update"])
    print(f"installing apache2 on {container_name}...")
    container.execute(["apt", "install", "-y","apache2"])
    print(f"apache2 installed on {container_name}\n")

updating www0...
installing apache2 on www0...
apache2 installed on www0

updating www1...
installing apache2 on www1...
apache2 installed on www1

updating www2...
installing apache2 on www2...
apache2 installed on www2



## 4. Testing your containers' webservers ( 20 points )
Fetch each container's IP address using the `Instance.state()` function then use the python library `requests` to print out the HTTP status code.

In [18]:
# Write your code here
import requests
from pylxd import Client
client = Client()
container_names = ["www0", "www1", "www2"]
for container_name in container_names:
    container = client.containers.get(container_name)
    container_state = container.state()
    ip = None
    for network in container_state.network.values():
        for address in network["addresses"]:
            if address["family"] == "inet":
                ip = address["address"]
                break
    if ip is None:
        print(f"{container_name}: No ip found")
    else:
        try:
            response = requests.get(f"http://{ip}")
            print(f"{container_name}: http {response.status_code}")
        except requests.ConnectionError:
            print(f"{container_name}: Connection fail.")

www0: Connection fail.
www1: Connection fail.
www2: Connection fail.


In [ ]:
# The connection failures in the code were caused by a network conflict between eth0 and lxdbr0, 
# as both were connected to the same network. To address this, I created a new bridge network (lxdbr1) 
# and moved the containers to it, eliminating the conflict. However, despite enabling DHCP for lxdbr1, 
# the containers still did not obtain IP addresses. I attempted to restart the containers to reflect the new settings,
# but the issue persisted, leading to continued connection failures. This suggests that the containers may not have 
# been fully configured to adapt to the new network settings.


## Mounting directories inside of a container's mount namespace

```
mkdir -p /srv/data
lxc config device add www0 data disk source=/srv/data/ path=/srv/data
lxc exec www0 -- mount
```

```
root@o8horizt-server:~# lxc exec www0 -- mount
/dev/vda1 on / type ext4 (rw,relatime)
none on /dev type tmpfs (rw,relatime,size=492k,mode=755,uid=1000000,gid=1000000)
proc on /proc type proc (rw,nosuid,nodev,noexec,relatime)
sysfs on /sys type sysfs (rw,relatime)
...
...
/dev/vda1 on /srv/data type ext4 (rw,relatime)
```